In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pwd

'/content'

In [ ]:
try:
    import geopandas as gpd
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        !pip install geopandas --quiet
    else:
        print('geopandas not found, please install via conda in your environment')

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import sqlite3
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, Normalizer, OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score, adjusted_mutual_info_score, calinski_harabasz_score, davies_bouldin_score
import joblib
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
# Load dataset | Read the GeoPackage/CSV file into a GeoDataFrame/DataFrame
DATASET_GPKG = "/content/drive/MyDrive/IPAUA_Maz/dataset/combined_v2.gpkg"
DATASET_CSV = "/content/drive/MyDrive/IPAUA_Maz/dataset/combined_v2.csv"
LAYER_NAME = "spatial_Joins"
CHUNK_SIZE = 10000
RANDOM_SAMPLE = False
SAMPLE_FRACTION = 0.005
DROP_COLS = ['fid', 'geom', 'BU', 'Slope', 'Amenity', 'Zone', 'CH4', 'CO', 'HCHO', 'NO2', 'O3', 'SO2']

# Define parameters
NO_OF_CLUSTERS = 3
BATCH_SIZE = 1000
N_ITER = 100
MODEL_VERSION = "v2"

# File Paths
MODEL_FILE = f"/content/drive/MyDrive/IPAUA_Maz/models/kmeans_model_{MODEL_VERSION}.pkl"
BestAreas_zone_4 = "/content/drive/MyDrive/IPAUA_Maz/models/best_areas_zone_4.csv"
BestAreas_zone_9 = "/content/drive/MyDrive/IPAUA_Maz/models/best_areas_zone_9.csv"
METRICS = f"/content/drive/MyDrive/IPAUA_Maz/models/kmeans_metrics_{MODEL_VERSION}.csv"

In [ ]:
# Load dataset | Reading CSV file in chunks
def read_csv_in_chunks(csv_path, chunk_size=10000, random_sample=False, sample_fraction=0.1, drop_columns=None):
    # Get the total number of rows
    total_rows = sum(1 for _ in open(csv_path)) - 1  # Subtract 1 for header row

    print(f"Total rows to read: {total_rows}")

    chunks = []
    offset = 0

    if random_sample:
        sample_size = int(total_rows * sample_fraction)
        random_offsets = np.random.choice(total_rows, sample_size, replace=False)
        random_offsets.sort()
        total_rows = sample_size

    while offset < total_rows:
        if random_sample:
            start = random_offsets[offset]
            end = start + chunk_size if start + chunk_size < total_rows else total_rows
            skip_rows = list(range(1, start + 1)) + list(range(end + 1, total_rows + 1))
            chunk = pd.read_csv(csv_path, skiprows=skip_rows, nrows=chunk_size)
        else:
            chunk = pd.read_csv(csv_path, skiprows=range(1, offset + 1), nrows=chunk_size)

        # Drop specified columns if drop_columns list is provided
        if drop_columns:
            chunk = chunk.drop(columns=drop_columns)

        chunks.append(chunk)

        offset += chunk_size
        print(f"\rRead {offset} rows", end='', flush=True)

    # Concatenate all chunks into a single DataFrame
    full_df = pd.concat(chunks, ignore_index=True)
    # Drop duplicates
    full_df = full_df.drop_duplicates()

    return full_df


# Load dataset | Reading .gpkg file in chunks
def read_gpkg_in_chunks(gpkg_path, layer_name, chunk_size=10000, random_sample=False, sample_fraction=0.1, drop_columns=None):
    # Connect to the GeoPackage using sqlite3
    conn = sqlite3.connect(gpkg_path)

    # Get the total number of rows
    query_count = f"SELECT COUNT(*) FROM {layer_name}"
    total_rows = pd.read_sql(query_count, conn).iloc[0, 0]

    print(f"Total rows to read: {total_rows}")

    chunks = []
    offset = 0

    if random_sample:
        sample_size = int(total_rows * sample_fraction)
        random_offsets = np.random.choice(total_rows, sample_size, replace=False)
        random_offsets.sort()
        total_rows = sample_size

    while offset < total_rows:
        if random_sample:
            start = random_offsets[offset]
            end = start + chunk_size if start + chunk_size < total_rows else total_rows
            query = f"SELECT * FROM {layer_name} LIMIT {end - start} OFFSET {start}"
        else:
            query = f"SELECT * FROM {layer_name} LIMIT {chunk_size} OFFSET {offset}"

        chunk = pd.read_sql(query, conn)

        # Drop specified columns if drop_columns list is provided
        if drop_columns:
            chunk = chunk.drop(columns=drop_columns)

        # Convert the DataFrame chunk to a GeoDataFrame
        gdf_chunk = gpd.GeoDataFrame(
            chunk, geometry=gpd.points_from_xy(chunk['Longitude'], chunk['Latitude']), crs="EPSG:4326"
        )
        chunks.append(gdf_chunk)

        offset += chunk_size
        print(f"\rRead {offset} rows", end='', flush=True)

    conn.close()

    # Concatenate all chunks into a single GeoDataFrame
    full_gdf = pd.concat(chunks, ignore_index=True)
    # Drop duplicates
    full_gdf = full_gdf.drop_duplicates()

    return full_gdf


def preprocess_data(data, features, use_normalizer=False):
    # Separate numeric and categorical features
    numeric_features = data[features].select_dtypes(include=[np.number]).columns.tolist()
    categorical_features = data[features].select_dtypes(include=['object', 'category']).columns.tolist()

    # Impute missing values for numeric features
    tqdm.pandas(desc="Imputing missing values for numeric features")
    imputer_numeric = SimpleImputer(strategy="mean")
    X_numeric_imputed = imputer_numeric.fit_transform(data[numeric_features].progress_apply(lambda x: x))

    # Scale numeric features
    tqdm.pandas(desc="Scaling numeric features")
    scaler = StandardScaler()
    X_numeric_scaled = scaler.fit_transform(X_numeric_imputed)

    if use_normalizer:
        tqdm.pandas(desc="Normalizing numeric features")
        normalizer = Normalizer()
        X_numeric_normalized = normalizer.fit_transform(X_numeric_scaled)
        X_numeric = X_numeric_normalized
    else:
        X_numeric = X_numeric_scaled

    # Impute missing values for categorical features
    tqdm.pandas(desc="Imputing missing values for categorical features")
    imputer_categorical = SimpleImputer(strategy="most_frequent")
    X_categorical_imputed = imputer_categorical.fit_transform(data[categorical_features].progress_apply(lambda x: x))

    # Encode categorical features
    tqdm.pandas(desc="Encoding categorical features")
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    X_categorical_encoded = encoder.fit_transform(X_categorical_imputed)

    # Combine numeric and categorical features
    X_combined = np.hstack((X_numeric, X_categorical_encoded))

    # return X_combined, data
    return X_combined


# Calculate haversine distance
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance


# Function to encode amenities and calculate distance to LandUse
def encode_LandUse(data, LandUse_coords):
    data['Distance_to_LandUse'] = data.apply(lambda row: calculate_distance_to_nearest_LandUse(row['Latitude'], row['Longitude'], LandUse_coords), axis=1)
    return data

# Function to calculate distance to nearest LandUse
def calculate_distance_to_nearest_LandUse(latitude, longitude, LandUse_coords):
    min_distance = float('inf')
    for _, LandUse_coords in LandUse_coords.iterrows():
        distance = haversine_distance(latitude, longitude, LandUse_coords['Latitude'], LandUse_coords['Longitude'])
        if distance < min_distance:
            min_distance = distance
    return min_distance


# Perform K-means clustering
def perform_clustering(X_scaled, n_clusters=3, batch_size=100000, n_iter=10000):
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, batch_size=batch_size, n_init="auto")

    # Create a progress bar
    for i in tqdm(range(n_iter), desc="Clustering"):
        kmeans.partial_fit(X_scaled)

    clusters = kmeans.predict(X_scaled)
    return kmeans, clusters


# Visualize clusters
def visualize_clusters(X_scaled, clusters):
    # Progress bar setup
    steps = ['PCA Transformation', 'Plotting']
    pbar = tqdm(total=len(steps), desc="Visualization Steps")

    # Step 1: PCA Transformation
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    pbar.update(1)  # Update progress bar

    # Step 2: Plotting
    plt.figure(figsize=(10, 8))
    sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=clusters, palette="viridis")
    plt.title("Clusters of Urban Agriculture Areas")
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.legend(title="Cluster")
    plt.show()
    pbar.update(1)  # Update progress bar

    pbar.close()


# Select best areas
def select_best_areas(data, bounds):
    zone_data = data[
        (data["Latitude"] >= bounds["latitude_min"])
        & (data["Latitude"] <= bounds["latitude_max"])
        & (data["Longitude"] >= bounds["longitude_min"])
        & (data["Longitude"] <= bounds["longitude_max"])
    ]
    return zone_data


# Calculate suitability score
def calculate_suitability_score(row):
    acceptable_levels = {
        # "CH4": 1906.388,
        # "CO": 1e-3,
        # "HCHO": 1e-5,
        # "NO2": 5.3e-5,
        # "O3": 7e-5,
        # "SO2": 7.5e-5,
        "GHI": 0
    }

    weights = {
        "soil_moisture": 0.2,
        "NDBI": 0.1,
        # "BU": 0.1,
        "Roughness": 0.1,
        # "Slope": 0.1,
        "NDVI": 0.2,
        "LST": 0.1,
        "UHI": 0.1,
        "UTFVI": 0.1,
        "NDWI": 0.1,
        "SAVI": 0.1,
        "Distance_to_Amenities": -0.2,
        "GHI": 0.15,
        # "CH4": -0.05,
        # "CO": -0.1,
        # "HCHO": -0.1,
        # "NO2": -0.1,
        # "O3": -0.1,
        # "SO2": -0.1
    }

    def adjust_for_acceptable_levels(feature, value, acceptable_level):
        if feature == "soil_moisture":
            if value < 20 or value > 150:
                return 0
        if feature == "NDBI":
            if value < -0.2 or value > 0.1:
                return 0
        # if feature == "BU":
        #     if value < -0.5 or value > 0.1:
        #         return 0
        if feature == "Roughness":
            if value < 0 or value > 10:
                return 0
        # if feature == "Slope":
        #     # if value < 0 or value > 10:
        #     if value < 0 or value > 87:
        #         return 0
        if feature == "NDVI":
            if value < 0.2 or value > 1:
                return 0
        if feature == "LST":
            if value < 15 or value > 35:
                return 0
        if feature == "UHI":
            if value < -5 or value > 5:
                return 0
        if feature == "UTFVI":
            if value < -0.5 or value > 0.5:
                return 0
        if feature == "NDWI":
            if value < -0.2 or value > 0.5:
                return 0
        if feature == "SAVI":
            if value < 0 or value > 0.5:
                return 0
        if feature == "GHI" and value < 0:
            return 0
        if feature in acceptable_levels:
            return (1 - abs(value - acceptable_levels[feature]) / acceptable_levels[feature]) * weights[feature]
        return value * weights[feature]

    suitability_score = sum(
        adjust_for_acceptable_levels(feature, row.get(feature, 0), acceptable_levels.get(feature, None))
        for feature in weights
    )
    return suitability_score


# Calculate performance metrics
def calculate_performance_metrics(X_scaled, clusters, kmeans):
    metrics = {}
    metrics['Inertia'] = kmeans.inertia_
    metrics['Rand Index'] = adjusted_rand_score(X_scaled, clusters)
    metrics['Mutual Info'] = adjusted_mutual_info_score(X_scaled, clusters)
    metrics['CHI Score'] = calinski_harabasz_score(X_scaled, clusters)
    metrics['DBI Score'] = davies_bouldin_score(X_scaled, clusters)
    metrics['SILHOUETTE_score'] = silhouette_score(X_scaled, clusters)
    return metrics


# Save the model
def save_model(model, file_name):
    joblib.dump(model, file_name)
    print(f"Model saved to {file_name}")

In [ ]:
# Load dataset | Reading CSV file in chunks
# data = read_csv_in_chunks(DATASET_CSV, chunk_size=CHUNK_SIZE, random_sample=RANDOM_SAMPLE, sample_fraction=SAMPLE_FRACTION, drop_columns=DROP_COLS)

# Load dataset | Read the GeoPackage file into a GeoDataFrame
data = read_gpkg_in_chunks(DATASET_GPKG, LAYER_NAME, chunk_size=CHUNK_SIZE, random_sample=RANDOM_SAMPLE, sample_fraction=SAMPLE_FRACTION, drop_columns=DROP_COLS)

# Define features for preprocessing
features = [
    "soil_moisture",
    "NDBI",
    # "BU", # Dropped since BU is already present in LULC Classes Column
    "Roughness",
    # "Slope",
    "NDVI", "LST", "UHI", "UTFVI", "NDWI", "SAVI", "lulc_classes", "LandUse", "GHI",
    # "CH4", "CO", "HCHO", "NO2", "O3", "SO2", # AIR QUIALITY
    "Longitude", "Latitude"
]

# Define preferred land use
preferred_land_use = [
    'allotment', 'brownfield', 'farmland', 'farmyard', 'flowerbed',
    'forest', 'grass', 'greenhouse_horticulture', 'meadow', 'orchard',
    'plant_nursery', 'residential', 'village_green'
]

Total rows to read: 1973361
Read 1980000 rows

In [ ]:
data.head()

,soil_moisture,NDBI,Roughness,NDVI,LST,UHI,UTFVI,NDWI,SAVI,lulc_classes,LandUse,GHI,Longitude,Latitude,geometry
0,125.331552,-0.129746,0.0,0.545336,28.333197,-2.657156,-0.199289,-0.567614,0.206486,4,grass,919.186730,9.240700,45.419989,POINT (9.24070 45.41999)
1,130.649125,-0.091416,0.0,0.705981,27.473296,-3.061812,-0.236826,-0.602588,0.327870,4,grass,560.302878,9.240341,45.420168,POINT (9.24034 45.42017)
2,130.643567,-0.112200,0.0,0.705981,27.473296,-3.061812,-0.236826,-0.579211,0.327870,4,grass,726.454937,9.240520,45.420168,POINT (9.24052 45.42017)
3,195.133153,-0.129746,0.0,0.545336,28.333197,-2.657156,-0.199289,-0.560563,0.206486,4,grass,644.406945,9.240700,45.420168,POINT (9.24070 45.42017)
4,132.716402,-0.129746,0.0,0.545336,28.333197,-2.657156,-0.199289,-0.621918,0.206486,4,grass,1146.273321,9.240879,45.420168,POINT (9.24088 45.42017)


In [ ]:
data = data.drop(columns=["geometry"])

In [ ]:
data.info()

In [ ]:
# Preprocess data
X_scaled = preprocess_data(data, features)

In [ ]:
# Encode land use and calculate distance to preferred land use
land_use_data = data.copy()
data_encoded = encode_LandUse(data, land_use_data)

In [ ]:
data_encoded.head()

In [ ]:
# Save DataFrame to a CSV file
data_encoded.to_csv("/content/drive/MyDrive/IPAUA_Maz/models/data_encoded.csv", index=False)
print(f"Data saved to {data_encoded}")

In [ ]:
# Perform K-means clustering
kmeans, clusters = perform_clustering(X_scaled, n_clusters=NO_OF_CLUSTERS, batch_size=BATCH_SIZE, n_iter=N_ITER)

In [ ]:
# Save the trained K-means model
save_model(kmeans, MODEL_FILE)

In [ ]:
# Calculate performance metrics
metrics = calculate_performance_metrics(X_scaled, clusters, kmeans)
for metric, value in metrics.items():
    print(f"{metric}: {value}")

metrics.to_csv(METRICS, index=False)
print(f"Metrics Saved to: {METRICS}")

In [ ]:
# Visualize clusters
visualize_clusters(X_scaled, clusters)

In [ ]:
# Define zone bounds
zone_4_bounds = {
    'latitude_min': 45.41993115091501,
    'latitude_max': 45.47215993917395,
    'longitude_min': 9.202965842470974,
    'longitude_max': 9.27280902210458
}

zone_9_bounds = {
    'latitude_min': 45.4801797942775,
    'latitude_max': 45.53632579458122,
    'longitude_min': 9.14322150832283,
    'longitude_max': 9.220977448916535
}

# Select the best areas for both zones
best_areas_zone_4 = select_best_areas(data_encoded, zone_4_bounds)
best_areas_zone_9 = select_best_areas(data_encoded, zone_9_bounds)

# Calculate suitability score for each area
best_areas_zone_4["Suitability_Score"] = best_areas_zone_4.apply(calculate_suitability_score, axis=1)
best_areas_zone_9["Suitability_Score"] = best_areas_zone_9.apply(calculate_suitability_score, axis=1)

# Perform K-means clustering for visualization
_, clusters_zone_4 = perform_clustering(X_scaled, n_clusters=NO_OF_CLUSTERS)
_, clusters_zone_9 = perform_clustering(X_scaled, n_clusters=NO_OF_CLUSTERS)

# Add clusters to the best areas DataFrame
best_areas_zone_4["Cluster"] = clusters_zone_4
best_areas_zone_9["Cluster"] = clusters_zone_9

# Plot the best areas for both zones
plt.figure(figsize=(10, 10))
plt.scatter(
    best_areas_zone_4["Longitude"],
    best_areas_zone_4["Latitude"],
    c=best_areas_zone_4["Cluster"],
    cmap="viridis",
    s=50,
    label='Zone 4'
)
plt.scatter(
    best_areas_zone_9["Longitude"],
    best_areas_zone_9["Latitude"],
    c=best_areas_zone_9["Cluster"],
    cmap="plasma",
    s=50,
    label='Zone 9'
)
plt.title("Best Areas for Urban Agriculture in Zones 4 and 9")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.colorbar(label="Cluster")
plt.show()

# Display the georeferences and relevant data for the best areas
_cols = [
            "soil_moisture", "NDBI",
            # "BU", # Dropped since BU is already present in LULC Classes Column
            "Roughness",
            # "Slope",
            "NDVI", "LST", "UHI", "UTFVI", "NDWI", "SAVI", "lulc_classes", "LandUse", "GHI",
            # "CH4", "CO", "HCHO", "NO2", "O3", "SO2", # AIR QUIALITY
            "Distance_to_LandUse", "Suitability_Score", "Longitude", "Latitude"
        ]

print("Best Areas in Zone 4")
print(
    best_areas_zone_4[
        _cols
    ]
)

print("\nBest Areas in Zone 9")
print(
    best_areas_zone_9[
        _cols
    ]
)

In [ ]:
# Save DataFrame to a CSV file
best_areas_zone_4.to_csv(BestAreas_zone_4, index=False)
print(f"Data saved to {BestAreas_zone_4}")
best_areas_zone_9.to_csv(BestAreas_zone_9, index=False)
print(f"Data saved to {BestAreas_zone_9}")

In [ ]:
# Load zone 4 polygon data from shape file
FILE_PATH_ZONE_4 = "/content/drive/MyDrive/IPAUA_Maz/shape_files/zone_4/layers/POLYGON.shp"
zone_4_polygon_data = gpd.read_file(FILE_PATH_ZONE_4)

# Create the plot
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the zone polygon
zone_4_polygon_data.plot(ax=ax, alpha=0.5, edgecolor='k')

# Plot the best areas on the map
scatter = ax.scatter(best_areas_zone_4['Longitude'], best_areas_zone_4['Latitude'], c=best_areas_zone_4['Cluster'], cmap='viridis', s=50)
plt.title('Best Areas for Urban Agriculture in Zone 4, Milan')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.colorbar(scatter, label='Cluster')

# Show the plot
plt.show()

In [ ]:
# Load zone 9 polygon data from shape file
FILE_PATH_ZONE_9 = "/content/drive/MyDrive/IPAUA_Maz/shape_files/zone_9/layers/POLYGON.shp"
zone_9_polygon_data = gpd.read_file(FILE_PATH_ZONE_9)

# Create the plot
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the zone polygon
zone_9_polygon_data.plot(ax=ax, alpha=0.5, edgecolor='k')

# Plot the best areas on the map
scatter = ax.scatter(best_areas_zone_9['Longitude'], best_areas_zone_9['Latitude'], c=best_areas_zone_9['Cluster'], cmap='viridis', s=50)
plt.title('Best Areas for Urban Agriculture in Zone 9, Milan')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.colorbar(scatter, label='Cluster')

# Show the plot
plt.show()